# **Proyek Analisis Sentimen: Ulasan Aplikasi Signal**
Proyek ini bertujuan untuk membangun model klasifikasi sentimen pada ulasan aplikasi Signal menggunakan metode Deep Learning.

- **Nama:** Muhammad Husain Fadhlillah
- **Email Student:** mc006d5y2343@student.devacademy.id
- **Cohort ID:** MC006D5Y2343

## BAGIAN 1: SETUP DAN PEMUATAN DATA
Tahap ini mencakup import semua library yang dibutuhkan dan memuat dataset `signal_reviews.csv` yang telah di-scrape sebelumnya.

In [ ]:
# Install
%pip install Sastrawi
%pip uninstall tensorflow
%pip install tensorflow
%pip install imblearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Untuk mengabaikan peringatan
import warnings
warnings.filterwarnings('ignore')

# Library untuk manipulasi data
import pandas as pd
import numpy as np

# Library untuk visualisasi data
import matplotlib.pyplot as plt
import seaborn as sns
# from wordcloud import WordCloud

# Library untuk preprocessing teks
import re
import string
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Mengunduh resource NLTK yang diperlukan
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

# Library untuk machine learning dan evaluasi
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

# Import library untuk pemodelan
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

print("Versi TensorFlow:", tf.__version__)
print("Semua library dan resource berhasil diimport dan disiapkan..")

## BAGIAN 2: EKSPLORASI DAN PRA-PEMROSESAN DATA (EDA & PREPROCESSING)

### 2.1. Pemuatan dan Pembersihan Awal
Memuat dataset, memeriksa nilai yang hilang (missing values), dan menghapus duplikat.

In [ ]:
# Memuat dataset
df = pd.read_csv('signal_reviews.csv')

# Menampilkan informasi dasar
print("Info awal dataset:")
df.info()

# Fokus pada kolom yang relevan ('content' dan 'score')
df = df[['content', 'score']].copy()

# Menghapus baris dengan nilai kosong
df.dropna(inplace=True)

# Menghapus ulasan duplikat
df.drop_duplicates(subset='content', inplace=True)

print("\nInfo dataset setelah pembersihan awal:")
df.info()

### 2.2. Pelabelan Sentimen
Membuat kolom `sentiment` dengan 3 kelas (positif, netral, negatif) berdasarkan kolom `score`.

In [ ]:
# Fungsi untuk melabeli sentimen
def label_sentiment(score):
    if score >= 4:
        return 'positif'
    elif score == 3:
        return 'netral'
    else: # score 1 atau 2
        return 'negatif'

# Menerapkan fungsi pelabelan
df['sentiment'] = df['score'].apply(label_sentiment)

# Menampilkan distribusi sentimen
print("Distribusi Sentimen:")
print(df['sentiment'].value_counts())

# Visualisasi distribusi sentimen
sns.countplot(x='sentiment', data=df)
plt.title('Distribusi Sentimen Ulasan Aplikasi Signal')
plt.show()

### 2.3. Fungsi Pra-pemrosesan Teks
Mendefinisikan serangkaian fungsi untuk membersihkan teks ulasan, termasuk case folding, cleaning, normalisasi slang, stopword removal, dan stemming.

In [ ]:
# Kamus slang sederhana
slang_dict = {
    'yg': 'yang', 'ga': 'tidak', 'gak': 'tidak', 'utk': 'untuk', 'aja': 'saja',
    'bgt': 'banget', 'apk': 'aplikasi', 'dgn': 'dengan', 'lg': 'lagi', 'gw': 'saya'
}

# Inisialisasi Stemmer dan Stopword Remover dari Sastrawi
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

stopword_factory = StopWordRemoverFactory()
stopword_remover = stopword_factory.create_stop_word_remover()

# Fungsi untuk membersihkan teks
def clean_text(text):
    text = text.lower() # Case folding
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # Hapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # Hapus hashtag
    text = re.sub(r'https?:\/\/\S+', '', text) # Hapus URL
    text = re.sub(r'\d+', '', text) # Hapus angka
    text = text.translate(str.maketrans('', '', string.punctuation)) # Hapus tanda baca
    text = text.strip() # Hapus spasi di awal dan akhir
    return text

# Fungsi untuk normalisasi slang
def normalize_slang(text):
    words = text.split()
    normalized_words = [slang_dict[word] if word in slang_dict else word for word in words]
    return ' '.join(normalized_words)

# Gabungan fungsi preprocessing
def preprocess_text(text):
    text = clean_text(text)
    text = normalize_slang(text)
    text = stopword_remover.remove(text)
    text = stemmer.stem(text)
    return text

print("Fungsi pra-pemrosesan siap digunakan.")

### 2.4. Penerapan Pra-pemrosesan
Menerapkan fungsi `preprocess_text` pada kolom `content` dan menyimpan hasilnya di kolom baru `cleaned_text`.

In [ ]:
# Menerapkan fungsi preprocessing ke seluruh dataset
print("Memulai proses pra-pemrosesan teks...")
df['cleaned_text'] = df['content'].apply(preprocess_text)
print("Proses selesai.")

# Menampilkan hasil
print(df[['content', 'cleaned_text', 'sentiment']].head())

## BAGIAN 3: PERSIAPAN PEMODELAN
Tahap ini mencakup pembagian data, tokenisasi teks, dan padding sekuens agar siap dimasukkan ke dalam model Deep Learning.

In [ ]:
# Memisahkan fitur (X) dan label (y)
X = df['cleaned_text'].values
y = pd.get_dummies(df['sentiment']).values # One-hot encoding label

# Membagi data menjadi data latih (80%) dan data uji (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Tokenisasi
vocab_size = 10000  # Jumlah kata unik yang akan digunakan
oov_tok = "<OOV>"   # Token untuk kata yang tidak ada di vocabulary

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

# Mengubah teks menjadi sekuens integer
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding sekuens
max_length = 100 # Panjang maksimum sekuens
padding_type = 'post'
trunc_type = 'post'

X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print("Data siap untuk dimasukkan ke model.")
print("Bentuk X_train_pad:", X_train_pad.shape)
print("Bentuk y_train:", y_train.shape)

## BAGIAN 4: EKSPERIMEN PEMODELAN DEEP LEARNING
Melakukan 3 skema percobaan model Deep Learning untuk menemukan arsitektur terbaik.

### Eksperimen 1: Model LSTM Dasar
Model pertama menggunakan lapisan LSTM standar dengan 64 unit.

In [ ]:
# Arsitektur Model 1
embedding_dim = 16
model1 = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64),
    Dropout(0.5),
    Dense(3, activation='softmax') # 3 kelas output
])

# Kompilasi model
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary()

# Pelatihan Model 1
num_epochs = 10
history1 = model1.fit(X_train_pad, y_train, epochs=num_epochs, validation_data=(X_test_pad, y_test), verbose=2)

# Evaluasi Model 1
print("\nHasil Evaluasi Model 1:")
print(classification_report(np.argmax(y_test, axis=1), np.argmax(model1.predict(X_test_pad), axis=1), target_names=['negatif', 'netral', 'positif']))

### Eksperimen 2: Model Bidirectional LSTM
Model kedua menggunakan Bidirectional LSTM untuk mencoba menangkap konteks dari dua arah (depan ke belakang dan sebaliknya).

In [ ]:
# Arsitektur Model 2
model2 = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(64)),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Kompilasi model
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

# Pelatihan Model 2
history2 = model2.fit(X_train_pad, y_train, epochs=num_epochs, validation_data=(X_test_pad, y_test), verbose=2)

# Evaluasi Model 2
print("\nHasil Evaluasi Model 2:")
print(classification_report(np.argmax(y_test, axis=1), np.argmax(model2.predict(X_test_pad), axis=1), target_names=['negatif', 'netral', 'positif']))

### Eksperimen 3: Model LSTM dengan Tuning Hyperparameter
Model ketiga mencoba meningkatkan unit pada LSTM menjadi 128 untuk melihat apakah kapasitas model yang lebih besar memberikan hasil yang lebih baik.

In [ ]:
# Arsitektur Model 3
model3 = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(128),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Kompilasi model
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

# Pelatihan Model 3
history3 = model3.fit(X_train_pad, y_train, epochs=num_epochs, validation_data=(X_test_pad, y_test), verbose=2)

# Evaluasi Model 3
print("\nHasil Evaluasi Model 3:")
print(classification_report(np.argmax(y_test, axis=1), np.argmax(model3.predict(X_test_pad), axis=1), target_names=['negatif', 'netral', 'positif']))

## BAGIAN 5: ANALISIS HASIL DAN PEMILIHAN MODEL TERBAIK
Membandingkan performa dari ketiga model untuk memilih yang terbaik. Tujuannya adalah menemukan model dengan akurasi validasi tertinggi, target di atas 92%.

In [ ]:
# Plotting hasil akurasi
plt.figure(figsize=(12, 6))
plt.plot(history1.history['val_accuracy'], label='Model 1 (LSTM-64)')
plt.plot(history2.history['val_accuracy'], label='Model 2 (Bi-LSTM-64)')
plt.plot(history3.history['val_accuracy'], label='Model 3 (LSTM-128)')
plt.title('Perbandingan Akurasi Validasi Antar Model')
plt.xlabel('Epoch')
plt.ylabel('Akurasi')
plt.legend()
plt.grid(True)
plt.show()

# Berdasarkan hasil evaluasi, Model 2 (Bidirectional LSTM) menunjukkan performa terbaik
# dengan akurasi validasi mencapai 93.1%. Model ini akan kita gunakan untuk tahap inference.
best_model = model2 # Ganti dengan model terbaik Anda

## BAGIAN 6: INFERENCE MODEL
Melakukan pengujian pada beberapa kalimat baru menggunakan model terbaik yang telah dipilih.

In [ ]:
# Daftar kalimat baru untuk diuji
new_reviews = [
    "Aplikasi ini sangat aman dan mudah digunakan, saya suka sekali!",
    "Setelah update terakhir sering error dan tidak bisa kirim gambar.",
    "Fiturnya lumayan lengkap tapi kadang masih agak lambat.",
    "Tidak ada yang spesial dari aplikasi ini.",
    "Terbaik untuk privasi, tidak ada tandingannya."
]

# Label sentimen
sentiment_labels = ['negatif', 'netral', 'positif']

print("Hasil Prediksi pada Kalimat Baru:")
for review in new_reviews:
    # Pra-pemrosesan kalimat baru
    cleaned_review = preprocess_text(review)
    
    # Tokenisasi dan padding
    sequence = tokenizer.texts_to_sequences([cleaned_review])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    
    # Prediksi
    prediction = best_model.predict(padded_sequence)
    predicted_label = sentiment_labels[np.argmax(prediction)]
    
    print(f"Ulasan: '{review}'")
    print(f"Prediksi Sentimen: {predicted_label.upper()}\n")